# Census Data

Get the medium income by zipcode from 2011 to 2016

In [12]:
import requests
import pandas as pd
from multiprocessing import Pool

In [29]:
def get_zip_code_df_by_year(year):
    """Get median income by zipcode in Arizona by year"""
    url = 'https://api.census.gov/data/{0}'

    years_urls = {
        2011: {'url': url.format('2011/acs5'), 'name': 'B19013_001{0}'},
        2012: {'url': url.format('2012/acs5'), 'name': 'B19013_001{0}'},
        2013: {'url': url.format('2013/acs5'), 'name': 'B19013_001{0}'},
        2014: {'url': url.format('2014/acs5'), 'name': 'B19013_001{0}'},
        2015: {'url': url.format('2015/acs5'), 'name': 'B19013_001{0}'},
        2016: {
            'url': url.format('2016/acs/acs5/profile'),
            'name': 'DP03_0062{0}'
        }
    }

    url = years_urls[year]['url']
    name = years_urls[year]['name']

    keys = {'Total': 'E', 'Margin': 'M'}
    lowest_zip_code, highest_zip_code = 85001, 86556
    NAME = 'NAME,{0}'.format(name)
    query_params = {'for': 'zip code tabulation area:*'}

    dfs = []
    for col, key in keys.items():
        query_params['get'] = NAME.format(key)
        results = requests.get(url, params=query_params).json()
        columns = ['Name', col, 'Zip']
        df = pd.DataFrame(results[1:], columns=columns)
        df = df[['Zip', col]]
        df['Zip'] = df['Zip'].astype(int)
        df = df.rename(
            index=str,
            columns={'Zip': 'Zip', col: '{0} {1}'.format(col, year)}
        )
        dfs.append(df)

    med_income = dfs[0].merge(dfs[1], on='Zip')
    az_zips = (
        (med_income['Zip'] >= lowest_zip_code) &
        (med_income['Zip'] <= highest_zip_code)
    )
    az_med_income = med_income[az_zips].reset_index(drop=True)
    az_med_income.set_index('Zip', inplace=True)
    az_med_income.sort_index(inplace=True)
    return az_med_income

In [30]:
with Pool(processes=8) as pool:
    dfs = pool.map(get_zip_code_df_by_year, range(2011, 2017))
df = pd.concat(dfs, axis=1)
with open('median_income_by_zip_code.csv', 'w') as stream:
    df.to_csv(stream, index=True)

In [31]:
df

,Total 2011,Margin 2011,Total 2012,Margin 2012,Total 2013,Margin 2013,Total 2014,Margin 2014,Total 2015,Margin 2015,Total 2016,Margin 2016
Zip,,,,,,,,,,,,
85003,33013,4597,33244,4474,36390,6045,42661,7409,41038,6632,45615,6225
85004,25572,5049,25552,4997,26008,5960,29567,8372,30202,10995,34595,6008
85006,20856,1529,21006,1250,22957,2579,24983,2243,26317,1931,29903,2840
85007,33770,5879,30618,4395,28125,4024,27200,4054,28380,3357,27557,2852
85008,32412,2176,32003,1425,32109,1505,34581,1973,33481,1972,34677,1485
85009,26268,1501,24587,1854,22725,1196,22137,906,21719,1024,23302,1577
85012,52197,9075,51637,6965,49226,6995,53333,7883,58108,7945,55542,9236
85013,42936,2691,45707,2999,43576,3259,46920,3144,50247,4885,50182,3535
85014,43331,4502,42003,2495,41903,2210,42162,1736,43369,1870,43682,1900
